In [1]:
from bert_serving.client import BertClient

In [2]:
bert_client = BertClient()

In [3]:
import numpy as np
import pandas as pd

/Users/yitinglin/Projects/news-emotion-analysis/env/lib/python3.6/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [4]:
content_df = pd.read_csv('./data/Train_Dataset.csv')
label_df = pd.read_csv('./data/Train_Dataset_Label.csv')

In [5]:
df = content_df.merge(label_df, on='id')
df = df.dropna()

In [6]:
embedded = []
for text in df.content.values:
    embedded.append(bert_client.encode([text]))

/Users/yitinglin/Projects/news-emotion-analysis/env/lib/python3.6/site-packages/bert_serving/client/__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=512" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


In [7]:
len(embedded)

7265

In [8]:
embedded = np.array(embedded)

In [9]:
embedded.shape

(7265, 1, 768)

In [10]:
df.shape

(7265, 4)

In [12]:
word_vector = []
for i in range(embedded.shape[0]):
    word_vector.append(embedded[i][0])

In [13]:
word_vector = np.array(word_vector)
print(word_vector.shape)

(7265, 768)


In [15]:
pd.DataFrame(word_vector).to_csv('./train_word_vector.csv', header=None)

In [16]:
X = word_vector
y = df['label'].values

In [17]:
import tensorflow as tf

/Users/yitinglin/Projects/news-emotion-analysis/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/yitinglin/Projects/news-emotion-analysis/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/yitinglin/Projects/news-emotion-analysis/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(y)
y = tf.keras.utils.to_categorical(y)

ValueError: bad input shape (7265, 3)

In [34]:
model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation=tf.nn.relu, input_shape=(768,)),
        tf.keras.layers.Dense(32, activation=tf.nn.relu),
        tf.keras.layers.Dense(32, activation=tf.nn.relu),
        tf.keras.layers.Dense(3, activation='softmax')
    ])

In [35]:
LR = 1e-4
model.compile(
    tf.keras.optimizers.Adam(lr=LR),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [38]:
BATCH_SIZE = 32
model.fit(
    X,
    y,
    epochs=50,
    batch_size=BATCH_SIZE,
)

Epoch 1/50
7265/7265 [==============================] - 0s 31us/sample - loss: 0.4222 - acc: 0.8307
Epoch 2/50
7265/7265 [==============================] - 0s 26us/sample - loss: 0.4208 - acc: 0.8260
Epoch 3/50
7265/7265 [==============================] - 0s 27us/sample - loss: 0.4126 - acc: 0.8319
Epoch 4/50
7265/7265 [==============================] - 0s 26us/sample - loss: 0.4101 - acc: 0.8369
Epoch 5/50
7265/7265 [==============================] - 0s 30us/sample - loss: 0.4074 - acc: 0.8351
Epoch 6/50
7265/7265 [==============================] - 0s 28us/sample - loss: 0.3985 - acc: 0.8398
Epoch 7/50
7265/7265 [==============================] - 0s 26us/sample - loss: 0.3998 - acc: 0.8409
Epoch 8/50
7265/7265 [==============================] - 0s 26us/sample - loss: 0.3919 - acc: 0.8421
Epoch 9/50
7265/7265 [==============================] - 0s 30us/sample - loss: 0.3881 - acc: 0.8449
Epoch 10/50
7265/7265 [==============================] - 0s 29us/sample - loss: 0.3821 - acc: 0.8507

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2019)

In [42]:
y_train.shape

(5085, 3)

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2019)

In [45]:
model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation=tf.nn.relu, input_shape=(768,)),
        tf.keras.layers.Dense(32, activation=tf.nn.relu),
        tf.keras.layers.Dense(32, activation=tf.nn.relu),
        tf.keras.layers.Dense(3, activation='softmax')
    ])
LR = 1e-4
model.compile(
    tf.keras.optimizers.Adam(lr=LR),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [46]:
BATCH_SIZE = 32
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=BATCH_SIZE,
    callbacks=[early_stop]
)

Train on 5812 samples, validate on 1453 samples
Epoch 1/50
5812/5812 [==============================] - 0s 57us/sample - loss: 0.8199 - acc: 0.6564 - val_loss: 0.6618 - val_acc: 0.7330
Epoch 2/50
5812/5812 [==============================] - 0s 34us/sample - loss: 0.6417 - acc: 0.7326 - val_loss: 0.5987 - val_acc: 0.7522
Epoch 3/50
5812/5812 [==============================] - 0s 34us/sample - loss: 0.5943 - acc: 0.7497 - val_loss: 0.5794 - val_acc: 0.7571
Epoch 4/50
5812/5812 [==============================] - 0s 32us/sample - loss: 0.5722 - acc: 0.7591 - val_loss: 0.5633 - val_acc: 0.7674
Epoch 5/50
5812/5812 [==============================] - 0s 34us/sample - loss: 0.5539 - acc: 0.7669 - val_loss: 0.5523 - val_acc: 0.7729
Epoch 6/50
5812/5812 [==============================] - 0s 34us/sample - loss: 0.5424 - acc: 0.7713 - val_loss: 0.5480 - val_acc: 0.7708
Epoch 7/50
5812/5812 [==============================] - 0s 34us/sample - loss: 0.5331 - acc: 0.7756 - val_loss: 0.5429 - val_acc: 

In [47]:
test_df = pd.read_csv('./data/Test_Dataset.csv')

In [51]:
test_data = []
for content in test_df.content.values:
    test_data.append(bert_client.encode([str(content)])[0])

/Users/yitinglin/Projects/news-emotion-analysis/env/lib/python3.6/site-packages/bert_serving/client/__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=512" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


In [54]:
test_data = np.array(test_data)

In [55]:
test_data.shape

(7356, 768)

In [56]:
test_df.shape

(7356, 3)

In [61]:
pred = model.predict(test_data)
print(pred)

[[0.59444314 0.40047696 0.00507991]
 [0.00951869 0.44488338 0.545598  ]
 [0.00357998 0.7644521  0.23196794]
 ...
 [0.5376648  0.4588728  0.00346242]
 [0.6087117  0.3863722  0.00491608]
 [0.01484168 0.5477541  0.43740413]]


In [66]:
submission_df = pd.DataFrame()
submission_df['id'] = test_df['id'].values
submission_df['label'] = np.argmax(pred, axis=1)

In [68]:
submission_df.to_csv('./submission.csv', index=False)

In [70]:
pd.DataFrame(test_data).to_csv('./test_word_vector.csv')

In [71]:
from sklearn.metrics import f1_score

In [121]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(y_train.shape)

(5812, 3)


In [122]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

model = tf.keras.Sequential([
        tf.keras.layers.Reshape((64, 12), input_shape=(768,)),
        tf.keras.layers.Conv1D(32, 8, activation=tf.nn.relu, input_shape=(64, 12)),
        tf.keras.layers.Conv1D(32, 8, activation=tf.nn.relu),
        tf.keras.layers.MaxPooling1D(3),
        tf.keras.layers.Conv1D(64, 8, activation=tf.nn.relu),
        tf.keras.layers.Conv1D(64, 8, activation=tf.nn.relu),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(3, activation='softmax')
    ])
LR = 1e-4
model.compile(
    tf.keras.optimizers.Adam(lr=LR),
    loss='categorical_crossentropy',
    metrics=[f1],
)

In [123]:
BATCH_SIZE = 32
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=BATCH_SIZE,
    callbacks=[early_stop]
)

Train on 5812 samples, validate on 1453 samples
Epoch 1/50
5812/5812 [==============================] - 1s 231us/sample - loss: 0.8741 - f1: 0.4506 - val_loss: 0.7513 - val_f1: 0.6775
Epoch 2/50
5812/5812 [==============================] - 1s 164us/sample - loss: 0.7081 - f1: 0.7020 - val_loss: 0.6660 - val_f1: 0.7230
Epoch 3/50
5812/5812 [==============================] - 1s 166us/sample - loss: 0.6520 - f1: 0.7260 - val_loss: 0.6271 - val_f1: 0.7350
Epoch 4/50
5812/5812 [==============================] - 1s 165us/sample - loss: 0.6160 - f1: 0.7382 - val_loss: 0.6023 - val_f1: 0.7392
Epoch 5/50
5812/5812 [==============================] - 1s 169us/sample - loss: 0.5889 - f1: 0.7479 - val_loss: 0.5825 - val_f1: 0.7461
Epoch 6/50
5812/5812 [==============================] - 1s 175us/sample - loss: 0.5695 - f1: 0.7560 - val_loss: 0.5883 - val_f1: 0.7485
Epoch 7/50
5812/5812 [==============================] - 1s 173us/sample - loss: 0.5586 - f1: 0.7617 - val_loss: 0.5626 - val_f1: 0.7546


In [125]:
pred = model.predict(test_data)

In [126]:
pred

array([[0.4720546 , 0.51232606, 0.01561937],
       [0.03073281, 0.5135095 , 0.45575768],
       [0.00545582, 0.8614308 , 0.1331134 ],
       ...,
       [0.57019615, 0.42523158, 0.0045723 ],
       [0.76545376, 0.2305294 , 0.00401679],
       [0.03027752, 0.62384385, 0.34587863]], dtype=float32)

In [127]:
submission_df = pd.DataFrame()
submission_df['id'] = test_df['id'].values
submission_df['label'] = np.argmax(pred, axis=1)

In [128]:
submission_df.to_csv('./submission.csv', index=False)